<a href="https://colab.research.google.com/github/mon-project-repos/ReviewGenerator/blob/master/ReviewGenerator_OneHotNextWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preliminaries

## GPU Session

In [0]:
# set up session 
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)


import os
import tensorflow as tf
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

## Imports

In [0]:
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
import re

import nltk
# nltk.download('stopwords')
nltk.download('punkt')

from tensorflow.keras.preprocessing.sequence import pad_sequences

## Data

In [0]:
dta = pickle.load(open("gdrive/My Drive/dta_review.pkl",'rb'))

dta = dta.loc[dta['prod_total']>100]






print('Total',dta.shape)



corpus = dta['review_data']
dta = None

# NLP preprocessing

## User and Product Documents

In [0]:
len(corpus)
percent_validation = .2
last_train = int(len(corpus)*(1-percent_validation))
print(corpus[last_train-1])
corpus_train, corpus_validation = corpus[:last_train], corpus[last_train:]
corpus = None

In [0]:
corpus_train = (" end-of_review  ").join(corpus_train)
sents_train = nltk.sent_tokenize(corpus_train)
len(sents_train) 
#remove_filters='"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n' # don't remove anything as they may be emojis
max_vocab_words = 500
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_vocab_words+1,
                                                  lower=True, 
                                                  filters="",
                                                 oov_token="<OOV>",char_level=False)



tokenizer.fit_on_texts(sents_train)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(tokenizer.word_counts)

print(len(tokenizer.word_index))
print(len(tokenizer.word_counts))

## Create input sequences

In [0]:
maxlen_ngram = 30
review_sequences = []
for review in sents_train:
    tokenized_review = tokenizer.texts_to_sequences([review])[0] #map each word to word_index 
    #print(len(tokenized_review))
    for i in range(1, min( [maxlen_ngram,len(tokenized_review)])): # make ngrams
        review_sequences.append( tokenized_review[:i+1] )
        
# get max length 
max_review_len = max([len(x) for x in review_sequences])
print(max_review_len)
        


In [0]:
#
review_sequences = np.array(pad_sequences(review_sequences,
                                         maxlen=maxlen_ngram,
                                         padding='pre'))    
    
# generate set of last words in sequence
predictors, last_word = review_sequences[:,:-1] , review_sequences[:,-1]
last_word = tf.keras.utils.to_categorical(last_word, num_classes=max_vocab_words+1)


In [74]:
corpus_validation = (" end-of-review  ").join(corpus_validation)
sents_validation = nltk.sent_tokenize(corpus_validation)
review_sequences_validation = []

for review in sents_validation:
    
    tokenized_review = tokenizer.texts_to_sequences([review])[0]
    for i in  range(1, min([maxlen_ngram, len(tokenized_review)])): # make ngrams
        review_sequences_validation.append( tokenized_review[:i+1] )
        
# get max length 
max_review_len_validation = max([len(x) for x in review_sequences_validation])
print(max_review_len_validation)
        
    
review_sequences_validation = np.array(pad_sequences(review_sequences_validation,
                                         maxlen=maxlen_ngram,
                                         padding='pre', truncating='post'))    
    


30


In [0]:
# generate set of last words in sequence
predictors_validation, last_word_validation = review_sequences_validation[:,:-1] , review_sequences_validation[:,-1]
last_word_validation = tf.keras.utils.to_categorical(last_word_validation, num_classes=max_vocab_words+1)

# LSTM Model

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_vocab_words +1 , maxlen_ngram, input_length=max_review_len-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences = True)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(40))
model.add(tf.keras.layers.Dense(50, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(max_vocab_words +1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


In [0]:
class earlyStopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('acc')> 0.7):
            print("\n Training Accuracy is 0.9.  Stopping here")
            self.model.stop_training = True
         
callbacks = earlyStopCallback()

In [0]:
history = model.fit(predictors, last_word, 
                    epochs=20, 
                    verbose = 2, 
                    validation_data = (predictors_validation,last_word_validation),
#                    validation_split=0.2, 
                    batch_size = 512, 
                    callbacks = [callbacks])

In [0]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()

In [0]:
seed_text = "OMG!  "
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=maxlen_ngram-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)